In [17]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False
mpl.rcParams['font.sans-serif'] = ['SimHei']   #设置简黑字体
mpl.rcParams['axes.unicode_minus'] = False # 解决‘-’bug

csv_path = './605296.csv'

x = []
y = []
a = []
b = []
c = []
d = []
for i in range(100):
    x.append(0)
    y.append(0)
    a.append(0)
    b.append(0)
    c.append(0)
    d.append(0)
start = 0
end = 1
for i in range(100):
    df = pd.read_csv(csv_path)
    # 切片操作 [start:stop:step]，从索引5起每6步取一行数据
    df = df[start * 300:end * 300:2]
    start = start + 1
    end = end + 1
    date_time = pd.to_datetime(df.pop('bob'), format='%Y-%m-%d %H:%M:%S')

    #plot_cols = ['open', 'high', 'close']
    #plot_features = df[plot_cols]
    #plot_features.index = date_time
    #_ = plot_features.plot(subplots=True)

    #plot_features = df[plot_cols][:480]
    #plot_features.index = date_time[:480]
    #_ = plot_features.plot(subplots=True)

    #df.describe().transpose()

    column_indices = {name: i for i, name in enumerate(df.columns)}

    n = len(df)
    train_df = df[0:int(n * 0.7)]
    val_df = df[int(n * 0.7):int(n * 0.9)]
    test_df = df[int(n * 0.9):]

    num_features = df.shape[1]

    train_mean = train_df.mean()
    train_std = train_df.std()

    train_df = (train_df - train_mean) / train_std
    val_df = (val_df - train_mean) / train_std
    test_df = (test_df - train_mean) / train_std

    class WindowGenerator():
        def __init__(self, input_width, label_width, shift,
                     train_df=train_df, val_df=val_df, test_df=test_df,
                     label_columns=None):
            # 储存原始数据
            self.train_df = train_df
            self.val_df = val_df
            self.test_df = test_df

            # 处理标签页索引
            self.label_columns = label_columns
            if label_columns is not None:
                self.label_columns_indices = {name: i for i, name in
                                              enumerate(label_columns)}
            self.column_indices = {name: i for i, name in
                                   enumerate(train_df.columns)}

            # 处理窗口参数
            self.input_width = input_width
            self.label_width = label_width
            self.shift = shift

            self.total_window_size = input_width + shift

            self.input_slice = slice(0, input_width)
            self.input_indices = np.arange(self.total_window_size)[self.input_slice]

            self.label_start = self.total_window_size - self.label_width
            self.labels_slice = slice(self.label_start, None)
            self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

        def __repr__(self):
            return '\n'.join([
                f'Total window size: {self.total_window_size}',
                f'Input indices: {self.input_indices}',
                f'Label indices: {self.label_indices}',
                f'Label column name(s): {self.label_columns}'])


    w1 = WindowGenerator(input_width=24, label_width=1, shift=24,
                         label_columns=['open'])
    w2 = WindowGenerator(input_width=6, label_width=1, shift=1,
                         label_columns=['open'])


    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # slicing不会保留固定形状信息，因此需要手动设置形状
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels


    WindowGenerator.split_window = split_window
    # 保存三个数据切片和所有窗口的长度
    #example_window = tf.stack([np.array(train_df[:w2.total_window_size]),
    #                           np.array(train_df[100:100 + w2.total_window_size]),
    #                           np.array(train_df[200:200 + w2.total_window_size])])

    #example_inputs, example_labels = w2.split_window(example_window)

    #print('所有形状格式为: (批量数, 时间长度, 特征)')
    #print(f'窗口形状: {example_window.shape}')
    #print(f'输入形状: {example_inputs.shape}')
    #print(f'标签形状: {example_labels.shape}')

    #w2.example = example_inputs, example_labels


    def plot(self, model=None, plot_col='open', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n + 1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='输入', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='标签', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                            marker='X', edgecolors='k', label='预测',
                            c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('时间 [h]')


    WindowGenerator.plot = plot

    #w2.plot()

    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32, )

        ds = ds.map(self.split_window)

        return ds


    WindowGenerator.make_dataset = make_dataset


    @property
    def train(self):
        return self.make_dataset(self.train_df)


    @property
    def val(self):
        return self.make_dataset(self.val_df)


    @property
    def test(self):
        return self.make_dataset(self.test_df)


    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.train))
            # And cache it for next time
            self._example = result
        return result


    WindowGenerator.train = train
    WindowGenerator.val = val
    WindowGenerator.test = test
    WindowGenerator.example = example

    #for example_inputs, example_labels in w2.train.take(1):
        #print(f'输入形状 (批量数, 时间步长, 特征): {example_inputs.shape}')
        #print(f'标签形状 (批量数, 时间步长, 特征): {example_labels.shape}')

    single_step_window = WindowGenerator(
        input_width=1, label_width=1, shift=1,
        label_columns=['open'])
    single_step_window

    #for example_inputs, example_labels in single_step_window.train.take(1):
        #print(f'输入形状 (批量数, 时间步长, 特征): {example_inputs.shape}')
        #print(f'标签形状 (批量数, 时间步长, 特征): {example_labels.shape}')


    class Baseline(tf.keras.Model):
        def __init__(self, label_index=None):
            super().__init__()
            self.label_index = label_index

        def call(self, inputs):
            if self.label_index is None:
                return inputs
            result = inputs[:, :, self.label_index]
            return result[:, :, tf.newaxis]


    baseline = Baseline(label_index=column_indices['open'])

    baseline.compile(loss=tf.losses.MeanSquaredError(),
                     metrics=[tf.metrics.MeanAbsoluteError()])

    val_performance = {}
    performance = {}
    val_performance['Baseline'] = baseline.evaluate(single_step_window.val)
    performance['Baseline'] = baseline.evaluate(single_step_window.test, verbose=0)
    d[i] = val_performance['Baseline'][1]

    wide_window = WindowGenerator(
        input_width=24, label_width=24, shift=1,
        label_columns=['open'])

    wide_window

    #print('输入形状:', wide_window.example[0].shape)
    #print('输出形状:', baseline(wide_window.example[0]).shape)

    #wide_window.plot(baseline)

    linear = tf.keras.Sequential([
        tf.keras.layers.Dense(units=1)
    ])

    MAX_EPOCHS = 20


    def compile_and_fit(model, window, patience=2):
        # EarlyStopping 提前终止，避免过拟合
        # monitor 要检测的指标， val_loss 验证集上的损失
        # pationce 容忍度，如果检测指标在容忍度epochs之外还没有改善，则终止
        # mode 指的改善的方向。 min，表示val_loss 越小越好。 如果两次（pationce==2）之后，val_loss没有变得更小，则停止训练
        early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                          patience=patience,
                                                          mode='min')

        model.compile(loss=tf.losses.MeanSquaredError(),
                      optimizer=tf.optimizers.Adam(),
                      metrics=[tf.metrics.MeanAbsoluteError()])

        history = model.fit(window.train, epochs=MAX_EPOCHS,
                            validation_data=window.val,
                            callbacks=[early_stopping])
        return history


    history = compile_and_fit(linear, single_step_window)

    val_performance['Linear'] = linear.evaluate(single_step_window.val)
    performance['Linear'] = linear.evaluate(single_step_window.test, verbose=0)
    c[i] = val_performance['Linear'][1]

    #plt.bar(x=range(len(train_df.columns)),
    #        height=linear.layers[0].kernel[:, 0].numpy())
    #axis = plt.gca()
    #axis.set_xticks(range(len(train_df.columns)))
    #_ = axis.set_xticklabels(train_df.columns, rotation=90)

    dense = tf.keras.Sequential([
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=64, activation='relu'),
        tf.keras.layers.Dense(units=1)
    ])

    history = compile_and_fit(dense, single_step_window)

    val_performance['Dense'] = dense.evaluate(single_step_window.val)
    performance['Dense'] = dense.evaluate(single_step_window.test, verbose=0)
    
    y[i] = val_performance['Dense'][1]

    CONV_WIDTH = 3
    conv_window = WindowGenerator(
        input_width=CONV_WIDTH,
        label_width=1,
        shift=1,
        label_columns=['open'])

    conv_window

    multi_step_dense = tf.keras.Sequential([
        # 形状: (时间步, 特征数) => (时间 * 特征数)
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=1),
        # 重新添加时间轴
        # 形状: (输出数目) => (1, 输入数目)
        tf.keras.layers.Reshape([1, -1]),
    ])

    history = compile_and_fit(multi_step_dense, conv_window)

    IPython.display.clear_output()
    val_performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.val)
    performance['Multi step dense'] = multi_step_dense.evaluate(conv_window.test, verbose=0)
    b[i] = val_performance['Multi step dense'][1]

    conv_model = tf.keras.Sequential([
        tf.keras.layers.Conv1D(filters=32,
                               kernel_size=(CONV_WIDTH,),
                               activation='relu'),
        tf.keras.layers.Dense(units=32, activation='relu'),
        tf.keras.layers.Dense(units=1),
    ])

    history = compile_and_fit(conv_model, conv_window)

    IPython.display.clear_output()
    val_performance['Conv'] = conv_model.evaluate(conv_window.val)
    performance['Conv'] = conv_model.evaluate(conv_window.test, verbose=0)
    a[i] = val_performance['Conv'][1]

    LABEL_WIDTH = 24
    INPUT_WIDTH = LABEL_WIDTH + (CONV_WIDTH - 1)
    wide_conv_window = WindowGenerator(
        input_width=INPUT_WIDTH,
        label_width=LABEL_WIDTH,
        shift=1,
        label_columns=['open'])

    wide_conv_window

    lstm_model = tf.keras.models.Sequential([
        # 形状 [批量数, 时间步, 特征数] => [批量数, 时间步, 单元数]
        tf.keras.layers.LSTM(32, return_sequences=True),
        # 形状 => [批量数, 时间步, 特征数]
        tf.keras.layers.Dense(units=1)
    ])

    history = compile_and_fit(lstm_model, wide_window)

    IPython.display.clear_output()
    val_performance['LSTM'] = lstm_model.evaluate(wide_window.val)
    performance['LSTM'] = lstm_model.evaluate(wide_window.test, verbose=0)
    x[i] = val_performance['LSTM'][1]
sum = 0
for i in range(100):
    sum = sum + x[i]
print("LSTM平均误差{}".format(sum / 100))
sum2 = 0
for i in range(100):
    sum2 = sum2 + y[i]
print("Dense平均误差{}".format(sum2 / 100))
sum3 = 0
for i in range(100):
    sum3 = sum3 + a[i]
print("Conv平均误差{}".format(sum3 / 100))
sum4 = 0
for i in range(100):
    sum4 = sum4 + b[i]
print("Multi step dense平均误差{}".format(sum4 / 100))
sum5 = 0
for i in range(100):
    sum5 = sum5 + c[i]
print("Linear平均误差{}".format(sum5 / 100))
sum6 = 0
for i in range(100):
    sum6 = sum6 + d[i]
print("Basline平均误差{}".format(sum6 / 100))

1/1 [==============================] - 0s 90ms/step - loss: 0.4476 - mean_absolute_error: 0.5608
LSTM平均误差1.1073061931878327
Dense平均误差0.33054006047546863
Conv平均误差0.4101294995099306
Multi step dense平均误差0.4178814716637135
Linear平均误差1.8962954594939947
Basline平均误差0.2446501976624131
